In [2]:
import pandas as pd
import numpy as np
class KNN:
    def __init__(self,neighbour=2):
        self.neighbour = neighbour

    def cell_dector(self,row):
        count = 0
        for cell in self.missing_index:
            if(cell[0]==row):
                count = count+1
        return count
        
    def fill(self,dataset):
        self.dataset = dataset.copy()
        self.missing_index = np.argwhere(pd.isna(self.dataset))
        #print(self.missing_index)
        self.col = dataset.shape[1]
        distances = []
        for missing_cell in self.missing_index:
            unfilled_cell = self.cell_dector(missing_cell[0])
            filled_cell = self.col-unfilled_cell
            distance = []
            for i in range(self.dataset.shape[0]):
                if(i==missing_cell[0]):
                    continue
                row_a = self.dataset.iloc[missing_cell[0]]
                row_b = self.dataset.iloc[i]
                
                filled_cell_check = ~(row_a.isna() | row_b.isna())
                filled_cell_check.iloc[missing_cell[1]]=False
                perfect_a_row = row_a[filled_cell_check]
                perfect_b_row = row_b[filled_cell_check]

                if len(perfect_a_row) == 0:
                    continue
                
                dis_cal = np.sqrt((filled_cell/self.col)*np.sum((perfect_a_row-perfect_b_row)**2))
                distance.append((i,dis_cal))
            minimum_index = self.index_finder(distance)
            
            sum = 0
            check = 0
            for alok in minimum_index:
                if(pd.isna(self.dataset.iloc[alok,missing_cell[1]])):
                    continue
                    
                sum = sum+self.dataset.iloc[alok,missing_cell[1]]
                check = check+1
            if (check>0):
                fill_values = sum/check
                self.dataset.iloc[missing_cell[0],missing_cell[1]]=fill_values
            
        return self.dataset 
    def index_finder(self, distance):
        index = []
        value = []
        for dist in distance:
            index.append(dist[0])
            value.append(dist[1])
        value = np.array(value)
        index = np.array(index)
        minimum_value = value.argsort()
        minimum_index = index[minimum_value][:self.neighbour]
        return minimum_index
    
            
            

        

In [3]:
data = pd.read_csv('poly_train.csv')

In [4]:
data.head(2)

,x0,x1,x2,x3,x4,x5,x6,y
0,38715.885944,32120.376626,-103.380879,-105.843969,191159.044192,1.239707,-0.873959,2.183619e+40
1,21988.811712,18001.816613,-260.939557,-236.418680,812765.277732,-1.308924,-0.072857,1.191237e+38


In [5]:
target = data['y']

In [7]:
data = data.iloc[:,:-1]

In [9]:
knn = KNN(3)
newe_dataset = knn.fill(data)

In [13]:
newe_dataset['y']=target

In [15]:
newe_dataset.to_csv('poly_train_new_dataset.csv',index=False)